In [ ]:
# !nvidia-smi

In [ ]:
import tensorflow as tf

In [ ]:
# Check if a GPU is available
if tf.test.gpu_device_name():
    print('Found GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU found, using CPU")


In [ ]:
#!/usr/bin/python3
# Author: Leo Cao
# Date: Jan 9, 2026

# Procedure:
# 1. Indentify cats and dogs
# 2. Predict from own images
# Pet Images Data Set from GitHub: https://github.com/laxmimerit/dog-cat-full-dataset/archive/refs/heads/master.zip
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
# import torch
# torch.cuda.is_available()

In [ ]:
# Load the data
# import keras
# from keras.src.legacy.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1. / 255)
# Note: target_size needs to be a tuple (height, width)
training_set = train_datagen.flow_from_directory(
    './PetImages/train',
    target_size=(50, 50),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# Show class indices
training_set.class_indices

In [ ]:
# Set up Convolutional Neural Network (CNN) Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Explicitly place operations on the GPU (optional, often automatic)
#with tf.device('/device:GPU:0'):

# Build Sequential instance
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(50,50,3)))

# Add Convolutional Layer with 32 filters, 3 colors (RGB), strides 1.
model.add(Conv2D(32, (3, 3), activation = 'relu'))
# Add Pool Layer
model.add(MaxPool2D(pool_size = (2,2)))

# Add Convolutional Layer with 32 filters, 3 colors (RGB), strides 1.
model.add(Conv2D(32, (3, 3), activation = 'relu'))
# Add Pool Layer
model.add(MaxPool2D(pool_size = (2,2)))

# Flattening Layer
model.add(Flatten())

# Fully Connection (FC) Layer
model.add(Dense(units = 128, activation = 'relu' ))
model.add(Dense(units = 1, activation = 'sigmoid' ))

In [ ]:
# Configure the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )
model.summary()

In [ ]:

# Then train the model
model.fit(training_set, epochs = 25)

In [ ]:
#accuracy on the training data
accuracy_train = model.evaluate(training_set)
print(accuracy_train)

In [ ]:
#accuracy on the training data
test_set = train_datagen.flow_from_directory(
    './PetImages/test',
    target_size=(50, 50),
    batch_size=32,
    class_mode='binary'
)
accuracy_test = model.evaluate(test_set)
print(accuracy_test)

In [ ]:
# Identify own jpg by loading single image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
# load cat and dog images
pic_cat = load_img('./cat.jpg', target_size=(50, 50))
pic_dog = load_img('./dog.jpg', target_size=(50, 50))

# Convert from image to array
pic_cat = img_to_array(pic_cat)
pic_dog = img_to_array(pic_dog)


# Normalization
pic_cat = pic_cat / 255
pic_dog = pic_dog / 255

# Convert to 3 dimension
pic_cat = pic_cat.reshape(1, 50, 50, 3)
pic_dog = pic_dog.reshape(1, 50, 50, 3)

result_cat = model.predict(pic_cat)
result_dog = model.predict(pic_dog)

print(result_cat)
print(result_dog)

In [ ]:
import tensorflow as tf

# Save the entire model to a json and h5 file
model.save('cat_dog_classifier_0215.keras')

print("Model saved successfully to cat_dog_classifier_architecture.keras")

In [ ]:
# make prediction on multiple images
import matplotlib as mlp
font2 = {'family' : 'TeX Gyre Pagella',
'weight' : 'normal',
'size'   : 20,
}
mlp.rcParams['font.family'] = 'TeX Gyre Pagella'
mlp.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

a = [i for i in range(1,10)]
fig = plt.figure(figsize=(10,10))

for i in a:
    img_name = str(i)+'.jpg'
    img_ori = load_img(img_name, target_size = (50, 50))
    img = img_to_array(img_ori)
    img = img.astype('float32') / 255
    img = img.reshape(1, 50, 50, 3)
    result = model.predict(img)
    img_ori = load_img(img_name, target_size = (250, 250))
    plt.subplot(3,3,i)
    plt.imshow(img_ori)
    plt.title (f'Dog: {result}' if result[0][0] == 1 else f'Cat: {result}')
plt.show()